# two_party_continus_input_binay_classification

## 資料品質檢查

### 設定資料路徑 & 參數

In [13]:
import os
guest, host = 9999, 10000
data_base = "/data/projects/fate/"

dense_data = {"name": "breast_hetero_guest", "namespace": f"experiment"}
dense_data_dir = os.path.join(data_base, "persistence/data/breast_hetero_guest.csv")

### 缺失值 & 欄位名

In [2]:
import pandas as pd
dense_df = pd.read_csv(dense_data_dir)
print(dense_df.isna().sum())
print(dense_df.head(5))

id        0
target    0
x0        0
x1        0
x2        1
x3        1
x4        2
x5        1
x6        1
x7        0
x8        0
x9        2
dtype: int64
    id  target        x0        x1        x2        x3        x4        x5  \
0  133       1  0.254879 -1.046633  0.209656  0.074214 -0.441366 -0.377645   
1  273       1 -1.142928 -0.781198 -1.166747 -0.923578  0.628230 -1.021418   
2  175       1 -1.451067 -1.406518       NaN -1.092337 -0.708765 -1.168557   
3  551       1 -0.879933  0.420589 -0.877527 -0.780484 -1.037534 -0.483880   
4  199       0  0.426758  0.723479  0.316885  0.287273  1.000835       NaN   

         x6        x7        x8        x9  
0 -0.485934  0.347072 -0.287570 -0.733474  
1 -1.111867 -0.959523 -0.096672 -0.121683  
2 -1.305831 -1.745063 -0.499499 -0.302893  
3 -0.555498 -0.768581  0.433960 -0.200928  
4  1.077099  1.053586  2.996525  0.961696  


## 上傳資料

In [3]:
from pipeline.backend.pipeline import PipeLine
pipeline_upload = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest)
partition = 4

pipeline_upload.add_upload_data(file=dense_data_dir,
                                table_name=dense_data["name"],             # table name
                                namespace=dense_data["namespace"],         # namespace
                                head=1, partition=partition)               # data info
pipeline_upload.upload(drop=1)

/data/projects/python/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%

2023-06-06 03:28:11.792 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306060328116154190

2023-06-06 03:28:11.800 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00


m2023-06-06 03:28:12.826 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-06 03:28:12.828 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:01
2023-06-06 03:28:13.846 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-06-06 03:28:14.860 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-06-06 03:28:15.875 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-06-06 03:28:16.913 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:05
2023-06-06 03:28:17.945 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:06
2023-06-06 

### 建構 Training pipeline 範例

使用 `pipeline` 模塊來構建聯邦學習流程

In [4]:
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, DataTransform, Intersection, HeteroSecureBoost, Evaluation
from pipeline.interface import Data

實例化 `pipeline` 並設定 `initiator` 和 `roles`:

    - initiator: 
        * role: guest
        * party: 9999
    - roles:
        * guest: 9999
        * host: 10000
    

In [5]:
pipeline = PipeLine() \
        .set_initiator(role='guest', party_id=guest) \
        .set_roles(guest=guest, host=host)

使用 `Reader` 模塊來讀取資料, 注意這邊如果有 `K 個 Party` 參與則有 `K 個 Reader`

In [6]:
reader_0 = Reader(name="reader_0")
# set guest parameter
reader_0.get_party_instance(role='guest', party_id=guest).component_param(
    table={"name": "breast_hetero_guest", "namespace": "experiment"})
# set host parameter
reader_0.get_party_instance(role='host', party_id=host).component_param(
    table={"name": "breast_hetero_host", "namespace": "experiment"})

使用 `DataTransform` 模塊來讀取資料, 注意這邊如果有 `K 個 Party` 參與則有 `K 個 DataTransform`

`DataTransform` 負責資料前處理( 設定目標欄位名稱, 補缺值, 替換 outliers )

In [7]:
data_transform_0 = DataTransform(name="data_transform_0")
# set guest parameter
data_transform_0.get_party_instance(role='guest', party_id=guest).component_param(
    with_label=True, label_name='target', label_type='int', 
    missing_fill=True, missing_fill_method="designated", default_value=0.0,
    outlier_replace=False, outlier_replace_method=None, outlier_replace_value=0.0
)

data_transform_0.get_party_instance(role='host', party_id=[host]).component_param(
    with_label=False,
    missing_fill=True, missing_fill_method="designated", default_value=0.0,
    outlier_replace=False, outlier_replace_method=None, outlier_replace_value=0.0
)

使用 `Intersection` 模塊來達到對齊雙方相同顧客 ID, 且不會洩漏非相同顧客的 ID

In [8]:
intersect_0 = Intersection(name="intersect_0")

現在使用 `HeteroSecureBoost` 模塊. 用以下的參數來構建樹模型

In [9]:
hetero_secureboost_0 = HeteroSecureBoost(name="hetero_secureboost_0",
                                         num_trees=5,
                                         bin_num=16,
                                         task_type="classification",
                                         objective_param={"objective": "cross_entropy"},
                                         encrypt_param={"method": "iterativeAffine"},
                                         tree_param={"max_depth": 3})


最後, 為了檢驗好壞 使用 `Evaluation` 模塊來驗證好壞

In [10]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

上一個 component 的 output 是下一個 component 的 input

    - data_transform_0 吃 reader_0 的 output
    - intersect_0 吃 data_transform_0 的 output
    - hetero_secureboost_0 吃 intersect_0 的 output
    - evaluation_0 吃 hetero_secureboost_0 的 output (預測值)

記得用 `pipeline.compile()` 來打包整串流程
使用 `pipeline.fit()` 來開始進行訓練

In [11]:
pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(intersect_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(hetero_secureboost_0, data=Data(train_data=intersect_0.output.data))
pipeline.add_component(evaluation_0, data=Data(data=hetero_secureboost_0.output.data))
pipeline.compile()
pipeline.fit()

2023-06-06 03:28:27.588 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306060328213040790

2023-06-06 03:28:27.596 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-06-06 03:28:28.606 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
2023-06-06 03:28:29.616 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:02
2023-06-06 03:28:30.626 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:03
m2023-06-06 03:28:31.642 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-06 03:28:31.643 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2023-06-06 03:28:32.658 | INFO     | pipeline.utils.invo

ValueError: Job is failed, please check out job 202306060328213040790 by fate board or fate_flow cli

當訓練結束後, 模型會用來做預測. 使用者可以自由選擇要不要儲存此次 `pipeline` 以方便未來重複使用
使用 `pipeline.dump(pipeline_saved_path)` 來完成儲存

In [ ]:
pipeline.dump("pipeline_saved/pipeline_saved.pkl");

### 建構 Inference pipeline 範例

首先, 使用 `PipeLine.load_model_from_file` load `pkl` 檔

部署 Inference 需要的模塊, 在這邊是 `data_transform_0`, `intersect_0`, `hetero_secureboost_0`

In [ ]:
pipeline = PipeLine.load_model_from_file('pipeline_saved/pipeline_saved.pkl')
pipeline.deploy_component([pipeline.data_transform_0, pipeline.intersect_0, pipeline.hetero_secureboost_0]);

接著, 部署 `Reader` 模塊 `reader_1` 來讀取新data

In [ ]:
reader_1 = Reader(name="reader_1")
reader_1.get_party_instance(role="guest", party_id=guest).component_param(table={"name": "breast_hetero_guest", "namespace": "experiment"})
reader_1.get_party_instance(role="host", party_id=host).component_param(table={"name": "breast_hetero_host", "namespace": "experiment"})

最後, 部署新的 `Evaluation` 來衡量 predict ( Inference ) 的表現

In [ ]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

整合所有模塊

In [ ]:
predict_pipeline = PipeLine()
predict_pipeline.add_component(reader_1)\
                .add_component(pipeline, 
                               data=Data(predict_input={pipeline.data_transform_0.input.data: reader_1.output.data}))\
                .add_component(evaluation_0, data=Data(data=pipeline.hetero_secureboost_0.output.data));


預測!

In [12]:
predict_pipeline.predict()

NameError: name 'predict_pipeline' is not defined

用 `pipeline.get_component('evaluation_0').get_summary()` 

來取得 `evaluation_0` 模塊的資訓儲並存成 json 檔

In [ ]:
import json
data_base = "/data/projects/fate/"
metadata_saved_dir = os.path.join(data_base, "persistence/metadata/two_party_continus_input_binay_classification.json")
metedata = json.dumps(pipeline.get_component('evaluation_0').get_summary(), indent=4)

with open(metadata_saved_dir, "w") as json_file:
    json_file.write(metedata)
                                  
print(f"Write in metadata_saved_dir : {metadata_saved_dir} \n {metedata}")